# Import data and library

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
!pip install stop_words

In [39]:
import re
import string
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
import pandas as pd
import numpy as np

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Đọc dữ liệu từ các file CSV
datatrain = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/XuLyNgonNguTuNhien/data/datatrain.csv")
datavalid = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/XuLyNgonNguTuNhien/data/datavalid.csv")
datatest = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/XuLyNgonNguTuNhien/data/datatest.csv")

# Gộp các bộ dữ liệu thành một DataFrame
combined_data = pd.concat([datatrain, datavalid, datatest], ignore_index=True)

# Xáo trộn dữ liệu
combined_data_shuffled = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Sử dụng train_test_split để tách lại thành các bộ dữ liệu
datatrain, temp = train_test_split(combined_data_shuffled, test_size=0.4, random_state=42)
datavalid, datatest = train_test_split(temp, test_size=0.5, random_state=42)

# In thông tin về kích thước của từng bộ dữ liệu
print("Kích thước bộ dữ liệu datatrain:", datatrain.shape)
print("Kích thước bộ dữ liệu datavalid:", datavalid.shape)
print("Kích thước bộ dữ liệu datatest:", datatest.shape)


Kích thước bộ dữ liệu datatrain: (28065, 5)
Kích thước bộ dữ liệu datavalid: (9355, 5)
Kích thước bộ dữ liệu datatest: (9356, 5)


# Process data

In [41]:
datatrain = datatrain.drop("Unnamed: 0", axis=1)
datavalid = datavalid.drop("Unnamed: 0", axis=1)
datatest = datatest.drop("Unnamed: 0", axis=1)

In [42]:
datatrain.reset_index(drop=True, inplace=True)
datavalid.reset_index(drop=True, inplace=True)
datatest.reset_index(drop=True, inplace=True)

In [43]:
#@title list emoticons
EMOTICONS = {
    u"uwu":"uwu face",
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

In [44]:
#@title Process all
from stop_words import get_stop_words
import re
import string
# Lấy danh sách stopword tiếng Việt
stop_words = get_stop_words('vi')

def processAll(text):
  # punctuation
  cleaned_sentence = text.translate(str.maketrans('', '', string.punctuation))

  # stop word
  words = cleaned_sentence.split()
  filtered_words = [word for word in words if word.lower() not in stop_words]
  stopword_done = ' '.join(filtered_words)

  # emoji
  emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
  emoji_done = emoji_pattern.sub(r'', stopword_done)

  # Emoticon
  emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
  emoticon_done = emoticon_pattern.sub(r'', emoji_done)

  # Url
  url_pattern = re.compile(r'https?://\S+|www\.\S+')
  url_done = url_pattern.sub(r'', emoticon_done)

  return url_done



In [45]:
for i in range(len(datatrain)):
  datatrain["Content"][i] = processAll(str(datatrain["Content"][i]))
for i in range(len(datavalid)):
  datavalid["Content"][i] = processAll(str(datavalid["Content"][i]))
for i in range(len(datatest)):
  datatest["Content"][i] = processAll(str(datatest["Content"][i]))

<ipython-input-45-0b14f0cf05d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatrain["Content"][i] = processAll(str(datatrain["Content"][i]))
<ipython-input-45-0b14f0cf05d9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datavalid["Content"][i] = processAll(str(datavalid["Content"][i]))
<ipython-input-45-0b14f0cf05d9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatest["Content"][i] = processAll(str(datatest["Content"][i]

In [46]:
# @title Take length text of row > 10
datatrainlite = datatrain[datatrain["Content"].apply(lambda x: len(str(x).split()) >= 10)]
datavalidlite = datavalid[datavalid["Content"].apply(lambda x: len(str(x).split()) >= 10)]
datatestlite = datatest[datatest["Content"].apply(lambda x: len(str(x).split()) >= 10)]

In [47]:
datatrainlite.reset_index(drop=True, inplace=True)
datavalidlite.reset_index(drop=True, inplace=True)
datatestlite.reset_index(drop=True, inplace=True)

In [48]:
datatrainlite['label'] = datatrainlite[['NEG', 'NEU', 'POS']].idxmax(axis=1)
datavalidlite['label'] = datavalidlite[['NEG', 'NEU', 'POS']].idxmax(axis=1)
datatestlite['label'] = datatestlite[['NEG', 'NEU', 'POS']].idxmax(axis=1)

<ipython-input-48-5e792fe78a7c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatrainlite['label'] = datatrainlite[['NEG', 'NEU', 'POS']].idxmax(axis=1)
<ipython-input-48-5e792fe78a7c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datavalidlite['label'] = datavalidlite[['NEG', 'NEU', 'POS']].idxmax(axis=1)
<ipython-input-48-5e792fe78a7c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [49]:
datatrainlite.drop(['NEG', 'NEU', 'POS'], axis=1, inplace=True)
datavalidlite.drop(['NEG', 'NEU', 'POS'], axis=1, inplace=True)
datatestlite.drop(['NEG', 'NEU', 'POS'], axis=1, inplace=True)

<ipython-input-49-98f3701f07d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatrainlite.drop(['NEG', 'NEU', 'POS'], axis=1, inplace=True)
<ipython-input-49-98f3701f07d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datavalidlite.drop(['NEG', 'NEU', 'POS'], axis=1, inplace=True)
<ipython-input-49-98f3701f07d1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datatestlite.drop(['NEG', 'NEU', 'POS'], axis=1, inplace=True)


# Transform

In [50]:
# train
train_sentences = datatrainlite['Content'].values
labels_train = datatrainlite['label'].values
# valid
valid_sentences = datavalidlite['Content'].values
labels_valid = datavalidlite['label'].values
#valid
test_sentences = datatestlite['Content'].values
labels_test = datatestlite['label'].values


In [51]:
labels_train = pd.DataFrame(labels_train, columns=["labels"])
labels_valid = pd.DataFrame(labels_valid, columns=["labels"])
labels_test = pd.DataFrame(labels_test, columns=["labels"])

In [52]:
#One-hot encode the lab
from keras.utils.np_utils import to_categorical
labels_train.loc[labels_train['labels'] == 'NEG', 'LABEL'] = 0
labels_train.loc[labels_train['labels'] == 'NEU', 'LABEL'] = 1
labels_train.loc[labels_train['labels'] == 'POS', 'LABEL'] = 2


labels_train = to_categorical(labels_train['LABEL'], num_classes=3)


In [53]:
#One-hot encode the lab
from keras.utils.np_utils import to_categorical
labels_valid.loc[labels_valid['labels'] == 'NEG', 'LABEL'] = 0
labels_valid.loc[labels_valid['labels'] == 'NEU', 'LABEL'] = 1
labels_valid.loc[labels_valid['labels'] == 'POS', 'LABEL'] = 2


labels_valid = to_categorical(labels_valid['LABEL'], num_classes=3)


In [54]:
labels_test.loc[labels_test['labels'] == 'NEG', 'LABEL'] = 0
labels_test.loc[labels_test['labels'] == 'NEU', 'LABEL'] = 1
labels_test.loc[labels_test['labels'] == 'POS', 'LABEL'] = 2

labels_test = to_categorical(labels_test['LABEL'], num_classes=3)


# Model

In [55]:
#@title Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'


In [56]:
#@title tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
valid_sequences = tokenizer.texts_to_sequences(valid_sentences)
valid_padded = pad_sequences(valid_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)



In [57]:
#@title model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 100)          300000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              84480     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 24)                3096      
                                                                 
 dense_3 (Dense)             (None, 3)                 75        
                                                                 
Total params: 387,651
Trainable params: 387,651
Non-trainable params: 0
_________________________________________________________________


In [58]:
num_epochs = 50
history = model.fit(train_padded, labels_train,
                    epochs=num_epochs, verbose=1,
                    validation_data=(valid_padded, labels_valid))

Epoch 1/50
204/204 [==============================] - 25s 100ms/step - loss: 0.6592 - accuracy: 0.7169 - val_loss: 0.5102 - val_accuracy: 0.7857
Epoch 2/50
204/204 [==============================] - 9s 43ms/step - loss: 0.4357 - accuracy: 0.8308 - val_loss: 0.5327 - val_accuracy: 0.7879
Epoch 3/50
204/204 [==============================] - 4s 21ms/step - loss: 0.3458 - accuracy: 0.8743 - val_loss: 0.5956 - val_accuracy: 0.7861
Epoch 4/50
204/204 [==============================] - 6s 28ms/step - loss: 0.2849 - accuracy: 0.8955 - val_loss: 0.6106 - val_accuracy: 0.7675
Epoch 5/50
204/204 [==============================] - 5s 23ms/step - loss: 0.2355 - accuracy: 0.9192 - val_loss: 0.6844 - val_accuracy: 0.7701
Epoch 6/50
204/204 [==============================] - 4s 21ms/step - loss: 0.2039 - accuracy: 0.9278 - val_loss: 0.7257 - val_accuracy: 0.7706
Epoch 7/50
204/204 [==============================] - 6s 28ms/step - loss: 0.1659 - accuracy: 0.9464 - val_loss: 0.7994 - val_accuracy: 0.75

# Predict

In [59]:
pred_test = model.predict(test_padded)
pred_test

68/68 [==============================] - 1s 9ms/step


array([[2.7977552e-02, 4.2661131e-06, 9.7201812e-01],
       [9.9991882e-01, 4.7364709e-05, 3.3863169e-05],
       [9.9999738e-01, 6.6291761e-08, 2.5605345e-06],
       ...,
       [1.0863724e-03, 3.3664917e-05, 9.9887997e-01],
       [6.2709639e-04, 3.8975955e-05, 9.9933392e-01],
       [9.9998045e-01, 1.8778082e-06, 1.7641025e-05]], dtype=float32)

In [60]:
pred_test[0]

array([2.7977552e-02, 4.2661131e-06, 9.7201812e-01], dtype=float32)

In [61]:
arr_pred_test = []
labels = ['neg', 'neu', 'pos']
for i in range(len(pred_test)):
  rs = labels[np.argmax(pred_test[i])]
  arr_pred_test.append(rs)

In [62]:
predicted_labels = []
for row in labels_test:
    index_of_one = np.argmax(row)  # Tìm chỉ số của giá trị 1 trong dòng
    predicted_label = labels[index_of_one]  # Lấy nhãn tương ứng với chỉ số
    predicted_labels.append(predicted_label)

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(arr_pred_test, predicted_labels)

0.7358402971216341

In [64]:
import numpy as np
txt = ["Cảm giác buồn và cô đơn luôn âm thầm tồn tại trong tâm hồn tôi, như những cơn mưa dầm dề trong bóng đêm không lối thoát."]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = ['neg', 'neu', 'pos']
print(pred, labels[np.argmax(pred)])

1/1 [==============================] - 0s 39ms/step
[[0.47057974 0.504999   0.02442129]] neu
